This tests the accuracy of a differential operator on a polynomial of the given order. If the operator is working to said order, then the `Actual` and `Theoretical` outputs should match (except possibly at the edges.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [54]:
# Grid parameters.
nh_min = 16
refRatio = 2
cfInter = int(nh_min / refRatio)

# Time integration parameters.
deriv = 'CD'
# RK = 4
order = 4

This cell contains the relevant switch information.

In [55]:
# TimeIntFunc, DiffFunc = TT.SolverSwitch(deriv, RK)

This cell instantiates the Grid and the Physics objects.

In [56]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)) + int(nh_min / refRatio))
omega.AddPatch(refRatio, finehalf)

This cell runs a test on the derivative operator on a polynomial which matches the given order.

In [57]:
TT.DerivPolyTest(omega, deriv, order, deriv = 0)

-1.0
xValsR: [-0.0625  0.125  -0.25    0.5    -1.    ]
xValsL: [-2.  2. -2.  2. -2.]
[ 1. -1.  1. -1.  1.]
YOU'RE USING THE UPDATE.
bounds: [-3.  -2.  -1.   0.   0.5  1. ]

-0.5
xValsR: [0. 0. 0. 0. 0.]
xValsL: [-0.0625  0.125  -0.25    0.5    -1.    ]
[ 0.0625 -0.125   0.25   -0.5     1.    ]
YOU'RE USING THE UPDATE.
bounds: [-2.  -1.   0.   0.5  1.   1.5]

0.5
xValsR: [0.0625 0.125  0.25   0.5    1.    ]
xValsL: [0. 0. 0. 0. 0.]
[0.0625 0.125  0.25   0.5    1.    ]
YOU'RE USING THE UPDATE.
bounds: [-1.5 -1.  -0.5 -0.   1.   2. ]

1.0
xValsR: [2. 2. 2. 2. 2.]
xValsL: [0.0625 0.125  0.25   0.5    1.    ]
[1. 1. 1. 1. 1.]
YOU'RE USING THE UPDATE.
bounds: [-1.  -0.5 -0.   1.   2.   3. ]

x:
[0.      0.0625  0.125   0.1875  0.25    0.3125  0.375   0.4375  0.5     0.53125 0.5625  0.59375 0.625   0.65625 0.6875  0.71875 0.75    0.78125 0.8125  0.84375 0.875   0.90625 0.9375  0.96875 1.     ]

Polynomial Function:
<p(x)> =
      5        4          3       2
0.2 x + 0.25 x + 0.3333 x + 0.5 x

In [58]:
coefs = 1. / (np.arange(order + 1) + 1)
coefs = np.append(1, coefs)[::-1]
P = np.poly1d(coefs) - 1
print(coefs)
print(P)
print('')

[0.2        0.25       0.33333333 0.5        1.         1.        ]
     5        4          3       2
0.2 x + 0.25 x + 0.3333 x + 0.5 x + 1 x



In [43]:
A = np.arange(10)
print(A)
print(A[:-1])
print('')

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8]

